# Building a Crew to Prepare for Meetings

In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Install Dependencies

Uncomment the following lines to install the required packages.

In [3]:
# Use the animation for pip install
# loader = LoadingAnimation()
# loader.start("Installing")
%pip install -r requirements.txt -q
# loader.stop("Installation complete")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Create reusable loading animation class
import os
import sys
import time
import threading

class LoadingAnimation:
    def __init__(self):
        self.stop_event = threading.Event()
        self.animation_thread = None

    def _animate(self, message="Loading"):
        chars = "/—\\|"
        while not self.stop_event.is_set():
            for char in chars:
                sys.stdout.write('\r' + message + '... ' + char)
                sys.stdout.flush()
                time.sleep(0.1)
                if self.stop_event.is_set():
                    sys.stdout.write("\n")
                    break

    def start(self, message="Loading"):
        self.stop_event.clear()
        self.animation_thread = threading.Thread(target=self._animate, args=(message,))
        self.animation_thread.daemon = True
        self.animation_thread.start()

    def stop(self, completion_message="Complete"):
        self.stop_event.set()
        if self.animation_thread:
            self.animation_thread.join()
        print(f"\r{completion_message} ✓")



### Helper Functions

In [5]:
import dotenv
from dotenv import dotenv_values

# Define a fake `load_dotenv` function
def _load_dotenv(*args, **kwargs):
    env_path = kwargs.get('dotenv_path', '.env')  # Default to '.env'
    parsed_env = dotenv_values(env_path)

    # Manually set valid key-value pairs
    for key, value in parsed_env.items():
        if key and value:  # Check for valid key-value pairs
            os.environ[key] = value

dotenv.load_dotenv = _load_dotenv

### Initialization and Setup
Initial imports for the CrewAI Flow and Crew and setting up the environment

In [6]:
# Importing necessary libraries
import os
import yaml

# Importing Crew related components
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/conda/lib/python3.12/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator("website_url")
/opt/conda/lib/python3.12/site-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator` v

In [8]:
# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Load agent and task configurations from YAML files
with open('config/agents.yml', 'r') as f:
    agents_config = yaml.safe_load(f)

with open('config/tasks.yml', 'r') as f:
    tasks_config = yaml.safe_load(f)

# Define the agents for our meeting preparation crew
researcher = Agent(
    role=agents_config['researcher']['role'],
    goal=agents_config['researcher']['goal'],
    backstory=agents_config['researcher']['backstory'],
    verbose=True,
    tools=[SerperDevTool(), ScrapeWebsiteTool()]  # Web search and document analysis tools
)

analyst = Agent(
    role=agents_config['analyst']['role'],
    goal=agents_config['analyst']['goal'],
    backstory=agents_config['analyst']['backstory'],
    verbose=True,
)

# Define the tasks for our crew
research_task = Task(
    description=tasks_config['research_task']['description'],
    expected_output=tasks_config['research_task']['expected_output'],
    agent=researcher,
)

preparation_task = Task(
    description=tasks_config['preparation_task']['description'],
    expected_output=tasks_config['preparation_task']['expected_output'],
    agent=analyst,
    output_file="output/meeting_brief.md"
)

# Create the crew
meeting_prep_crew = Crew(
    agents=[researcher, analyst],
    tasks=[research_task, preparation_task],
    process=Process.sequential,
    verbose=True
)

In [9]:
# If inputs are not provided, ask for them
meeting_topic = input("Enter the meeting topic: ")
participants = input("Enter the meeting participants: ")

print(f"\nPreparing for meeting about: {meeting_topic}")
print(f"Participants: {participants}\n")

# Run the crew with the provided inputs
result = meeting_prep_crew.kickoff(inputs={
    "topic": meeting_topic,
    "participants": participants
})

print("\n\nMeeting brief has been saved to output/meeting_brief.md")
# Display the markdown content in a formatted way
from IPython.display import Markdown, display

# Display the raw result as formatted markdown
display(Markdown(result.raw))

2025-03-14 20:36:55,429 - 281472977881920 - llm.py-llm:388 - ERROR: LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable



Preparing for meeting about: A sprint retrospective
Participants: macy@okta.com

 
[2025-03-14 20:36:55][🚀 CREW 'CREW' STARTED, 5B2E0249-2974-407E-9F95-F97D04695806]: 2025-03-14 20:36:55.409544
 
[2025-03-14 20:36:55][📋 TASK STARTED: CONDUCT STRATEGIC INTELLIGENCE GATHERING ON THE UPCOMING MEETING ABOUT A SPRINT RETROSPECTIVE WITH MACY@OKTA.COM.
YOUR OBJECTIVE IS TO UNCOVER INSIGHTS THAT WILL GIVE OUR EXECUTIVE A COMPETITIVE ADVANTAGE:
1. TOPIC ANALYSIS:
   - CORE BUSINESS IMPLICATIONS OF A SPRINT RETROSPECTIVE
   - LATEST DEVELOPMENTS AND EMERGING TRENDS (WITHIN LAST 3 MONTHS)
   - COMPETITIVE LANDSCAPE AND MARKET POSITION

2. PARTICIPANT INTELLIGENCE:
   - PROFESSIONAL BACKGROUND AND CURRENT ROLE OF EACH PARTICIPANT
   - RECENT STATEMENTS, PUBLICATIONS, OR POSITIONS ON A SPRINT RETROSPECTIVE
   - PERSONAL COMMUNICATION STYLE AND DECISION-MAKING PATTERNS
   - RELATIONSHIPS WITH OTHER PARTICIPANTS (ALLIANCES, TENSIONS)

3. STRATEGIC CONTEXT:
   - IDENTIFY POTENTIAL HIDDEN AGENDAS OR U

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable